## Register a metadata schema

The registration of the metadata schema is usually done by the administrator of the system. The metadata schema is defined in a JSON file, which is then registered in the system. This registration is not usually done as part of running tests or capturing data

In [ ]:
import sys
print(sys.executable)
!pip list

# from ni.measurements.metadata.v1.client import MetadataClient

# with MetadataClient() as client:
    # schema_id = client.register_schema("sample_schema.json")
    # print(f"registered schema id: {schema_id}")

In [ ]:
import sys
print(sys.executable)

## Create the session metadata

Now that a schema has been registered we can create our metadata objects with the expected metadata.  Note, however, the operator's 'badge_number' - which is required in the schema - has not been specified.

In [ ]:
from ni.measurements.data.v1.data import Session
from ni.measurements.metadata.v1.metadata import Operator, TestStation, Software

operator = Operator(operator_name="Nick Beer")
test_station = TestStation(test_station_name="TestStation_12", attributes={"location": "USA"})
software = Software(product="Windows", version="10.0.19044", attributes={"license": "LIC_enterprise"})
session = Session(
    session_name="sample data publish session",
    operator=operator,
    test_station=test_station,
    schema_id=schema_id,
    software=[software],
    attributes={"session_file_path": "C:\\my_test_description.xlsx"}
)

## Create the session object

Now that we have the session metadata, we can create the session object. Note below that schema validation can fail if your metadata does not match what's specified in the schema.

In [ ]:
import grpc
from ni.measurements.data.v1.client import DataStoreClient

with DataStoreClient() as client:
    try:
        session_id = client.create_session(session)
    except grpc.RpcError as e:
        print("Failed to create session:")
        print(f"  {e.details()}")

    # add the required attribute to the operator object
    operator.attributes["badge_number"] = "emp-128256"
    # change the station location to be invalid
    test_station.attributes["location"] = "Texas"

    try:
        session_id = client.create_session(session)
    except grpc.RpcError as e:
        print("Failed to create session:")
        print(f"  {e.details()}")

    # fix the station location
    test_station.attributes["location"] = "USA"
    # change the station name to be invalid (not matching the pattern defined in the schema)
    test_station.test_station_name = "TS_12"
    try:
        session_id = client.create_session(session)
    except grpc.RpcError as e:
        print("Failed to create session:")
        print(f"  {e.details()}")

    # fix the station name
    test_station.test_station_name = "TestStation_12"
    # change the software license to be invalid (not matching the pattern defined in the schema)
    software.attributes["license"] = "enterprise_LIC"

    try:
        session_id = client.create_session(session)
    except grpc.RpcError as e:
        print("Failed to create session:")
        print(f"  {e.details()}")

    # fix the software license to create the session
    software.attributes["license"] = "LIC_enterprise"
    session_id = client.create_session(session)
    print(f"created session id: {session_id}")

## Publish the data

Now that we have a valid session and schema, we can publish our data

In [ ]:
from datetime import datetime, timedelta, timezone
from ni.measurements.data.v1.client import DataStoreClient
from ni.measurements.data.v1.data import Measurement
from nitypes.waveform import AnalogWaveform
from nitypes.waveform import Timing
import numpy as np

with DataStoreClient() as client:
    name = "data publish sample"
    waveform = AnalogWaveform(
        sample_count=3,
        raw_data=np.array([1.0, 2.0, 3.0]),
        timing=Timing.create_with_regular_interval(
            timedelta(seconds=1e-3),
            datetime.now(timezone.utc)
        )
    )
    data = client.publish_double_analog_waveform(
        measurement=Measurement(name, session=session_id),
        name=name,
        description="An example of publishing data",
        waveform=waveform,
    )

    print(f"The published data id is {data.metadata().id}")

## Query the data

Now that we've got data in the database, we can query for it using our OData query api.

In [ ]:
from ni.measurements.data.v1.client import DataStoreClient
from ni.measurements.metadata.v1.client import MetadataClient

with DataStoreClient() as client:
    matches = client.query_data(odata_query=f"$filter=id eq {data.metadata().id}")
    data = matches[0]

    with MetadataClient() as metadata_client:
        operator = metadata_client.get_operator(data.metadata().operator_id)
        # badge_number is a custom attribute in the schema
        # It's not a standard attribute of the operator object
        badge_number = operator.badge_number
        print(f"operator {operator.operator_name}'s badge number is: {badge_number}")

    waveform = data.ValueAsDoubleAnalogWaveform()
    print(f"published data is: {waveform.raw_data}")